## Imports

In [1]:
import copy
import glob
import importlib
import itertools
import math
import os
import random
import shutil
import sys
from math import degrees, pi, radians
from os.path import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SimpleITK as sitk
import skimage.feature as skf
from skimage.morphology import ball

import cnn_builder as cbuild
import config
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.masks as masks
import niftiutils.registration as reg
import niftiutils.transforms as tr
from config import Config

%matplotlib inline

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [280]:
importlib.reload(config)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

## Pattern analysis

In [315]:
patient_id = "PK-08"
importlib.reload(lm)
paths = lm.get_paths(patient_id, target_dir, check_valid=False)

mask_dir, nii_dir, ct24_path, ct24_tumor_mask_path, ct24_liver_mask_path, \
mribl_art_path, mribl_pre_path, \
mribl_tumor_mask_path, mribl_liver_mask_path, \
mribl_enh_mask_path, mribl_nec_mask_path, \
mri30d_art_path, mri30d_pre_path, \
mri30d_tumor_mask_path, mri30d_liver_mask_path, \
mri30d_enh_mask_path, mri30d_nec_mask_path, \
ball_ct24_path, ball_mribl_path, ball_mri30d_path, \
ball_mask_path, ball_mribl_enh_mask_path, ball_mri30d_enh_mask_path, \
midlip_mask_path, ball_midlip_mask_path, \
highlip_mask_path, ball_highlip_mask_path = paths

### Build DataFrame

pattern_df = pd.DataFrame(columns=pattern_feat_cols)
lipresp_df = pd.DataFrame(columns=lipresp_cols)

for name in lipresp_cols:
    df.loc[:,"peripheral_coverage%"]=pd.Series(np.random.randn(len(df)), index=df.index)

In [425]:
pattern_feat_cols = ["enhancing_vol%", "lipcoverage_vol%",
                       "rim_enhancing%", "rim_lipiodol%",
                       "mribl_contrast", "mribl_variance", "mribl_homog",
                       "lip_contrast", "lip_variance", "lip_homog",
                       "peripheral_coverage%"]

lipresp_cols = []
for name in itertools.product(["base_", "low_", "mid_", "high_"], ["resp%", "prog%", "bl_enh%", "30d_enh%"]):
    lipresp_cols.append(name[0]+name[1])
    
# resp% is the % of BL enhancing tumor that becomes necrotic
# prog% is the % of BL necrotic tumor that becomes enhancing
# bl_enh% is the % of tumor that is enhancing on BL MR
# 30d_enh% is the % of tumor that is enhancing on 30d MR

In [421]:
for ix in df.index[14:]:
    patient_id = ix
    importlib.reload(lm)
    paths = lm.get_paths(patient_id, target_dir, check_valid=False)

    mask_dir, nii_dir, ct24_path, ct24_tumor_mask_path, ct24_liver_mask_path, \
    mribl_art_path, mribl_pre_path, \
    mribl_tumor_mask_path, mribl_liver_mask_path, \
    mribl_enh_mask_path, mribl_nec_mask_path, \
    mri30d_art_path, mri30d_pre_path, \
    mri30d_tumor_mask_path, mri30d_liver_mask_path, \
    mri30d_enh_mask_path, mri30d_nec_mask_path, \
    ball_ct24_path, ball_mribl_path, ball_mri30d_path, \
    ball_mask_path, ball_mribl_enh_mask_path, ball_mri30d_enh_mask_path, \
    midlip_mask_path, ball_midlip_mask_path, \
    highlip_mask_path, ball_highlip_mask_path = paths

    lips = lm.enhancing_to_nec(ball_ct24_path, ball_mribl_enh_mask_path, ball_mri30d_enh_mask_path, ball_mask_path)
    df.loc[ix,lipresp_cols] = np.round(np.array(lips)*100,1).flatten()

In [ ]:
pattern_df = pd.read_excel(C.data_xls_path, "Patterns")
lipresp_df = pd.read_excel(C.data_xls_path, "Lip-Resp")

In [394]:
importlib.reload(config)
C = config.Config()

In [62]:
[basename(fn) for fn in glob.glob(join(target_dir,"*"))].index('PK-02')

27

In [ ]:
importlib.reload(lm)
for fn in glob.glob(join(target_dir,"*")):
    patient_id = basename(fn)
    print(patient_id)
    #lm.spherize(patient_id, target_dir)
    pattern_df.loc[patient_id] = lm.get_row_entry(patient_id, target_dir)

In [424]:
pattern_df.to_excel(C.pattern_path, "Patterns")

## Create Ranked DICOMs

In [365]:
df.columns

Index(['enhancing_vol%', 'lipcoverage_vol%', 'rim_enhancing%', 'rim_lipiodol%',
       'mribl_contrast', 'mribl_variance', 'mribl_idm', 'lip_contrast',
       'lip_variance', 'lip_idm'],
      dtype='object')

In [296]:
importlib.reload(masks)
lm.write_ranked_imgs(df, "lipcoverage_vol%", "ct24_path", join(C.ranking_dir, "sparsity_ranking_ct24"),
                  overwrite=False, mask_type="ct24_tumor_mask_path", window="ct")
lm.write_ranked_imgs(df, "enhancing_vol%", "mribl_art_path", join(C.ranking_dir, "sparsity_ranking_mrbl"),
                  overwrite=False, mask_type="mribl_tumor_mask_path")

# make pointers instead?
lm.write_ranked_imgs(df, "rim_lipiodol%", "ct24_path", join(C.ranking_dir, "rim_ranking_ct24"),
                  overwrite=False, mask_type="ct24_tumor_mask_path", window="ct")
lm.write_ranked_imgs(df, "rim_enhancing%", "mribl_art_path", join(C.ranking_dir, "rim_ranking_mrbl"),
                  overwrite=False, mask_type="mribl_tumor_mask_path")

lm.write_ranked_imgs(df, "lip_idm", "ct24_path", join(C.ranking_dir, "homogen_ranking_ct24"),
                  overwrite=False, mask_type="ct24_tumor_mask_path", window="ct")
lm.write_ranked_imgs(df, "mribl_idm", "mribl_art_path", join(C.ranking_dir, "homogen_ranking_mrbl"),
                  overwrite=False, mask_type="mribl_tumor_mask_path")

In [141]:
masks.create_dcm_with_masks(ball_ct24_path, [ball_ct24_path], save_dir, overwrite=True, window=window)

In [ ]:
lm.spherize(patient_id, target_dir)

In [317]:
importlib.reload(lm)

<module 'lipiodol_methods' from 'C:\\Users\\Clinton\\Documents\\Lipiodol\\lipiodol_methods.py'>

In [337]:
mrbl_enh = masks.get_mask(ball_mribl_enh_mask_path)[0]
mr30d_enh = masks.get_mask(ball_mri30d_enh_mask_path)[0]
mrbl_nec = masks.difference(ball_mask_path, ball_mribl_enh_mask_path)
mr30d_nec = masks.difference(ball_mask_path, ball_mri30d_enh_mask_path)

ct24 = hf.nii_load(ball_ct24_path)[0]

resp = mrbl_enh * mr30d_nec/mr30d_nec.max()
unresp = mrbl_enh * mr30d_enh/mr30d_enh.max()
prog = mrbl_nec * mr30d_enh/mr30d_enh.max()
stable = mrbl_nec * mr30d_nec/mr30d_nec.max()

## Figures

In [ ]:
lm.draw_unreg_fig(mribl_art_path, mribl_enh_mask_path, "D:\\Lipiodol\\Figures\\unreg\\MRIBL", 'b', 'mr')
lm.draw_unreg_fig(mri30d_art_path, mri30d_enh_mask_path, "D:\\Lipiodol\\Figures\\unreg\\MRI30d", 'r', 'mr')
lm.draw_unreg_fig(ct24_path, highlip_mask_path, "D:\\Lipiodol\\Figures\\unreg\\Lip", 'g', 'ct')

In [ ]:
lm.draw_reg_fig(ball_mribl_path, ball_mribl_enh_mask_path, "D:\\Lipiodol\\Figures\\MRIBL", 'b', 'mr')
lm.draw_reg_fig(ball_mri30d_path, ball_mri30d_enh_mask_path, "D:\\Lipiodol\\Figures\\MRI30d", 'r', 'mr')
lm.draw_reg_fig(ball_ct24_path, ball_highlip_mask_path, "D:\\Lipiodol\\Figures\\Lip", 'g', 'ct')

In [ ]:
# Figure for 24h CT
img,_ = hf.nii_load(ball_ct24_path)
blmask,_ = masks.get_mask(ball_mribl_enh_mask_path)
fumask,_ = masks.get_mask(ball_mri30d_enh_mask_path)

for sl in range(img.shape[-1]//4,img.shape[-1]*3//4,img.shape[-1]//12):
    plt.close()
    plt.imshow(img[...,sl], cmap='gray', vmin=30, vmax=250)
    FU=plt.contour(fumask[:,:,sl], colors='r', alpha=.4)
    plt.contour(blmask[:,:,sl], colors='b', alpha=.4)
    plt.contourf(fumask[:,:,sl], colors=[(0,0,0,0)]*6+[(1,0,0,.2)]) #look at the length of FU.allsegs
    plt.contourf(blmask[:,:,sl], colors=[(0,0,0,0)]*6+[(0,0,1,.2)])
    plt.axis('off')
    plt.savefig("D:\\Lipiodol\\Figures\\24hCT_%d.png" % sl, dpi=100, bbox_inches='tight')

## Extra

IVs = get_intensity_section(ball_ct24_path, ball_mask_path, params)
            if highest_I_V < I_V:
                best_params = params
                highest_I_V = I_V
                print(best_params)
theta1_best, phi1_best, dtheta_best, dz_best = best_params

In [ ]:
ball_IV = get_avg_ball_intensity(ball_ct24_path, ball_mask_path)
core_IV = get_avg_core_intensity(ball_ct24_path, ball_mask_path)

In [303]:
ball_IV, core_IV

(197.83461276161094, 209.89937589866474)

In [ ]:
importlib.reload(lm)
IVs = lm.calc_intensity_shells_angles(hf.nii_load(ball_ct24_path)[0], ball_mask_path)
IVs[IVs==0] = np.nan

samples = lm.fibonacci_sphere(2500, True, randomize=True)
samples = np.round(samples).astype(int)
s0 = samples[:,0]
s1 = samples[:,1]
for i in range(IVs.shape[-1]):
    print(np.nanmean(IVs[s0,s1,i]))

In [388]:
loopIVs = np.tile(IVs[:-1,:-1,:], (2,2,1))
loopIVs[180:,:,:] = np.nan

In [ ]:
for dtheta in range(30,180,20):
    print(dtheta)
    best_IV = core_IV
    worst_IV = core_IV
    for theta in range(0,180,20):
        for phi in range(180,480,20):
            for shell1 in range(4):
                for shell2 in range(shell1+1,5):
                    IV = np.nanmean(loopIVs[max(theta-dtheta,0) : theta+dtheta, phi-dtheta : phi+dtheta, shell1:shell2])
                    params = theta, phi % 360, shell1, shell2
                    if best_IV < IV:
                        best_params = params
                        best_IV = IV
                        
            IV = np.nanmean(loopIVs[max(theta-dtheta,0) : theta+dtheta, phi-dtheta : phi+dtheta, :5])
            params = theta, phi % 360
            if worst_IV > IV:
                worst_params = params
                worst_IV = IV
                
    print(best_params, round(best_IV,1))
    print(worst_params, round(worst_IV,1))

In [ ]:
importlib.reload(lm)
enhmask_density = lm.calc_intensity_shells_angles(masks.get_mask(ball_mribl_enh_mask_path)[0]+1, ball_mask_path)

enhmask_density = enhmask_density[...,:5]
enhmask_density[enhmask_density==0] = np.nan

samples = fibonacci_sphere(2500, True, randomize=True)
samples = np.round(samples).astype(int)
s0 = samples[:,0]
s1 = samples[:,1]
for i in range(enhmask_density.shape[-1]):
    print(np.nanmean(enhmask_density[s0,s1,i]))

In [ ]:
importlib.reload(lm)
enhmask_density = lm.calc_intensity_shells_angles(masks.get_mask(ball_mri30d_enh_mask_path)[0]+1, ball_mask_path)

enhmask_density = enhmask_density[...,:5]
enhmask_density[enhmask_density==0] = np.nan

samples = fibonacci_sphere(2500, True, randomize=True)
samples = np.round(samples).astype(int)
s0 = samples[:,0]
s1 = samples[:,1]
for i in range(enhmask_density.shape[-1]):
    print(np.nanmean(enhmask_density[s0,s1,i]))

In [ ]:
for i in range(enhmask_density.shape[-1]):
    print(np.nanmean(enhmask_density[s0,s1,i]))

In [ ]:
enhmask_density =

In [ ]:
conv_rate = np.zeros((181,360))